In [1]:
import numpy as np
import pandas as pd

from datetime import datetime as dt

# Notebook ReadME

In this notebook, I want to make a prediction on what the spread should be based on the various factors for that game.  The main variables of interest will be the team's respective elo ratings, home field advantage, and the team's qb ratings.  I want to see if I can build a predictive model given these features that accurately predicts what a spread should be.

We will start by exploring the continuous variables of interest range of values based on the spread.  Then, we'll explore how the categorical features affect the target variable. Finally, we will train a model and make predictions on the spread after we've performed feature selection.

To assess how accurate the model is, we will compare our predictions with the actual spreads to get a sense of what the average error of our predictions is.

In an ideal world, we'll be able to use this model to bet on real NFL games!

# Elo Dataset

In [2]:
elo = pd.read_csv('nfl_elo.csv')

# filter out all games before 1980
elo = elo[elo.season >= 1980]

# rename columns
elo.rename(columns = {'date': 'schedule_date', 'team1': 'team_home', 'team2': 'team_away',
                      'score1': 'score_home', 'score2': 'score_away',
                      'elo1_pre': 'home_elo_pre', 'elo2_pre': 'away_elo_pre'}, inplace = True)

# let's whittle down the variables to the ones we think will affect the spread
elo = elo[['schedule_date', 'team_home', 'team_away', 'home_elo_pre', 'away_elo_pre', 'elo_prob1', 'elo_prob2',
           'qbelo1_pre', 'qbelo2_pre', 'qbelo_prob1', 'qbelo_prob2', 'score_home', 'score_away']]

# turn string into datetime
elo.schedule_date = elo.schedule_date.apply(lambda d: dt.strptime(d, '%Y-%m-%d'))
elo.head()

,schedule_date,team_home,team_away,home_elo_pre,away_elo_pre,elo_prob1,elo_prob2,qbelo1_pre,qbelo2_pre,qbelo_prob1,qbelo_prob2,score_home,score_away
6555,1980-09-07,CIN,TB,1470.732,1460.770,0.606236,0.393764,1466.536871,1464.160691,0.561730,0.438270,12,17
6556,1980-09-07,MIN,ATL,1452.529,1427.176,0.627174,0.372826,1450.740840,1427.566588,0.621037,0.378963,24,23
6557,1980-09-07,KC,OAK,1471.194,1546.315,0.485439,0.514561,1481.293992,1549.788298,0.492797,0.507203,14,27
6558,1980-09-07,PHI,DEN,1554.510,1550.319,0.598278,0.401722,1550.986646,1544.033387,0.614952,0.385048,27,6
6559,1980-09-07,NE,CLE,1535.825,1523.143,0.609967,0.390033,1531.627434,1518.534675,0.599850,0.400150,34,17


In [3]:
elo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10540 entries, 6555 to 17094
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   schedule_date  10540 non-null  datetime64[ns]
 1   team_home      10540 non-null  object        
 2   team_away      10540 non-null  object        
 3   home_elo_pre   10540 non-null  float64       
 4   away_elo_pre   10540 non-null  float64       
 5   elo_prob1      10540 non-null  float64       
 6   elo_prob2      10540 non-null  float64       
 7   qbelo1_pre     10540 non-null  float64       
 8   qbelo2_pre     10540 non-null  float64       
 9   qbelo_prob1    10540 non-null  float64       
 10  qbelo_prob2    10540 non-null  float64       
 11  score_home     10540 non-null  int64         
 12  score_away     10540 non-null  int64         
dtypes: datetime64[ns](1), float64(8), int64(2), object(2)
memory usage: 1.1+ MB


# Spread Dataset

In [4]:
def team_to_city(team):
    team_cities = {'Arizona Cardinals': 'ARI', 
                   'Atlanta Falcons': 'ATL', 
                   'Baltimore Colts': 'IND',
                   'Baltimore Ravens': 'BAL', 
                   'Buffalo Bills': 'BUF', 
                   'Carolina Panthers': 'CAR',
                   'Chicago Bears': 'CHI', 
                   'Cincinnati Bengals': 'CIN', 
                   'Cleveland Browns': 'CLE',
                   'Dallas Cowboys': 'DAL', 
                   'Denver Broncos': 'DEN', 
                   'Detroit Lions': 'DET',
                   'Green Bay Packers': 'GB', 
                   'Houston Oilers': 'TEN', 
                   'Houston Texans': 'HOU',
                   'Indianapolis Colts': 'IND', 
                   'Jacksonville Jaguars': 'JAX', 
                   'Kansas City Chiefs': 'KC',
                   'Las Vegas Raiders': 'OAK', 
                   'Los Angeles Chargers': 'LAC', 
                   'Los Angeles Raiders': 'OAK',
                   'Los Angeles Rams': 'LAR', 
                   'Miami Dolphins': 'MIA', 
                   'Minnesota Vikings': 'MIN',
                   'New England Patriots': 'NE', 
                   'New Orleans Saints': 'NO', 
                   'New York Giants': 'NYG',
                   'New York Jets': 'NYJ', 
                   'Oakland Raiders': 'OAK', 
                   'Philadelphia Eagles': 'PHI',
                   'Phoenix Cardinals': 'ARI', 
                   'Pittsburgh Steelers': 'PIT', 
                   'San Diego Chargers': 'LAC',
                   'San Francisco 49ers': 'SF', 
                   'Seattle Seahawks': 'SEA', 
                   'St. Louis Cardinals': 'ARI',
                   'St. Louis Rams': 'LAR', 
                   'Tampa Bay Buccaneers': 'TB', 
                   'Tennessee Oilers': 'TEN',
                   'Tennessee Titans': 'TEN', 
                   'Washington Commanders': 'WSH',
                   'Washington Football Team': 'WSH', 
                   'Washington Redskins': 'WSH'}
    
    return team_cities[team]

In [5]:
spreads = pd.read_csv('spreadspoke_scores.csv')

# filter out all data before 1980
spreads = spreads[spreads.schedule_season >= 1980]

# rename teams
spreads.team_home = spreads.team_home.apply(lambda t: team_to_city(t))
spreads.team_away = spreads.team_away.apply(lambda t: team_to_city(t))

# turn string into date
spreads.schedule_date = spreads.schedule_date.apply(lambda d: dt.strptime(d, '%m/%d/%Y'))

# filter out unnecessary columns
spreads = spreads[['schedule_date', 'team_home', 'score_home', 'score_away', 'team_away', 'team_favorite_id', 
                   'spread_favorite', 'over_under_line']]

spreads.head()

,schedule_date,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line
2734,1980-09-07,BUF,17,7,MIA,MIA,-3.0,35
2735,1980-09-07,CIN,12,17,TB,TB,-1.0,34
2736,1980-09-07,GB,12,6,CHI,CHI,-6.0,35
2737,1980-09-07,KC,14,27,OAK,KC,-3.0,42
2738,1980-09-07,LAR,20,41,DET,LAR,-9.0,38


In [6]:
spreads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10782 entries, 2734 to 13515
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   schedule_date     10782 non-null  datetime64[ns]
 1   team_home         10782 non-null  object        
 2   score_home        10782 non-null  int64         
 3   score_away        10782 non-null  int64         
 4   team_away         10782 non-null  object        
 5   team_favorite_id  10782 non-null  object        
 6   spread_favorite   10782 non-null  float64       
 7   over_under_line   10782 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 758.1+ KB


In [7]:
df = pd.merge(elo, spreads, on = ['schedule_date', 'team_home', 'team_away', 'score_home', 'score_away'], how = 'left')

# organize df
df = df[['schedule_date', 'team_home', 'score_home', 'score_away', 'team_away', 'team_favorite_id', 
         'spread_favorite', 'over_under_line', 'home_elo_pre', 'away_elo_pre', 'elo_prob1', 'elo_prob2',
         'qbelo1_pre', 'qbelo2_pre', 'qbelo_prob1', 'qbelo_prob2']]
df

,schedule_date,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,home_elo_pre,away_elo_pre,elo_prob1,elo_prob2,qbelo1_pre,qbelo2_pre,qbelo_prob1,qbelo_prob2
0,1980-09-07,CIN,12,17,TB,TB,-1.0,34,1470.732000,1460.770000,0.606236,0.393764,1466.536871,1464.160691,0.561730,0.438270
1,1980-09-07,MIN,24,23,ATL,MIN,-4.0,42,1452.529000,1427.176000,0.627174,0.372826,1450.740840,1427.566588,0.621037,0.378963
2,1980-09-07,KC,14,27,OAK,KC,-3.0,42,1471.194000,1546.315000,0.485439,0.514561,1481.293992,1549.788298,0.492797,0.507203
3,1980-09-07,PHI,27,6,DEN,PHI,-3.0,38,1554.510000,1550.319000,0.598278,0.401722,1550.986646,1544.033387,0.614952,0.385048
4,1980-09-07,NE,34,17,CLE,NE,-6.0,38,1535.825000,1523.143000,0.609967,0.390033,1531.627434,1518.534675,0.599850,0.400150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10535,2022-01-23,TB,27,30,LAR,TB,-3.0,48,1681.797158,1632.341105,0.659005,0.340995,1649.547292,1615.165080,0.681440,0.318560
10536,2022-01-23,KC,42,36,BUF,KC,-2.5,54,1699.470145,1683.332367,0.614689,0.385311,1687.450699,1656.910634,0.652720,0.347280
10537,2022-01-30,KC,24,27,CIN,KC,-7.0,54.5,1713.932369,1569.498961,0.769519,0.230481,1700.324756,1559.717895,0.810417,0.189583
10538,2022-01-30,LAR,20,17,SF,LAR,-3.5,46,1651.615346,1621.465562,0.633608,0.366392,1635.265484,1631.632019,0.636093,0.363907
